In [ ]:
!pip install mediapipe

In [2]:
import numpy as np
import pandas as pd
import mediapipe as mp
import os
import cv2
from google.colab import files
import keras
from keras.preprocessing.image import ImageDataGenerator

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [35]:
# A-Z minus J and Z
alphabets = list("ABCDEFGHIKLMNOPQRSTUVWXY")
alphabets.append('space')
print(alphabets)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'space']


In [ ]:
# Install and set up Kaggle
!pip install -q kaggle

In [ ]:
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d signnteam/asl-sign-language-pictures-minus-j-z

In [ ]:
# unzip
!mkdir train
!unzip asl-sign-language-pictures-minus-j-z.zip -d train
!rm asl-sign-language-pictures-minus-j-z.zip

In [ ]:
temp_dataset = []

for label in alphabets:
    dataset_directory_prefix = '/content/drive/MyDrive/Colab Notebooks/datasets/SigNN Character Database/'
    for dirname, _, filenames in os.walk(dataset_directory_prefix + label):
        for filename in filenames:
            pathname = os.path.join(dirname, filename)

            with mp_hands.Hands(
                static_image_mode=True,
                max_num_hands=1,
                min_detection_confidence=0.5
            ) as hands:

                # image = cv2.flip(cv2.imread(pathname), 1)
                image = cv2.imread(pathname)
                result = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                landmark_vertices_xyz = []
                if not result.multi_hand_landmarks:
                    continue

                for l in result.multi_hand_landmarks[0].landmark:
                    landmark_vertices_xyz.append(l.x)
                    landmark_vertices_xyz.append(l.y)
                    landmark_vertices_xyz.append(l.z)

                temp_dataset.append((*landmark_vertices_xyz, label))
                
    print(f'imported: {label}')
        
landmark_vertices_xyz_label = []
for idx in range(21):
    for char in list('xyz'):
        vertex_label = char+str(idx)
        landmark_vertices_xyz_label.append(vertex_label)
            
print(*landmark_vertices_xyz_label)
        
dataset = pd.DataFrame(temp_dataset, columns=[*landmark_vertices_xyz_label, 'target'])
dataset.head()

In [37]:
dataset.to_csv('data.csv', index=False)

In [26]:
dataset = pd.read_csv("/content/data.csv")

In [39]:
from sklearn.model_selection import train_test_split

X = dataset.drop(['target'],axis=1).values
y = dataset['target'].values

# Choose your test size to split between training and testing sets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [40]:
X_train.shape, X_test.shape

((6417, 63), (2139, 63))

In [41]:
y_train
y_train.shape

(6417,)

In [42]:
import tensorflow as tf

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=63, input_shape=[63]),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=25, activation='softmax')
])

model.summary()

In [ ]:
model_2 = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True, activation='relu',  input_shape=[21, 3]),
    tf.keras.layers.LSTM(128, return_sequences=True, activation='relu'),
    tf.keras.layers.LSTM(64, return_sequences=False, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(25, activation='softmax')
])

model_2.summary()

In [50]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [51]:
y_train_encoded = pd.get_dummies(y_train, prefix='target')
y_test_encoded = pd.get_dummies(y_test, prefix='target')
y_train_encoded.head(3)

,target_A,target_B,target_C,target_D,target_E,target_F,target_G,target_H,target_I,target_K,...,target_Q,target_R,target_S,target_T,target_U,target_V,target_W,target_X,target_Y,target_space
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)

In [ ]:
history = model.fit(
    X_train,
    y_train_encoded,
    validation_data=(X_test, y_test_encoded),
    epochs=100,
    callbacks=[early_stop]
)

Epoch 1/100


In [ ]:
# reshape for LSTM model
X_train = X_train.reshape([6186, 21, 3])
X_test = X_test.reshape([2062, 21, 3])

history = model_2.fit(
    X_train,
    y_train_encoded,
    validation_data=(X_test, y_test_encoded),
    epochs=100,
    callbacks=[early_stop]
)

In [ ]:
model.save('/content/my_model')

In [ ]:
model_2.save('/content/my_model')

# Evaluate Model

In [ ]:
model.evaluate(X_test, y_test_encoded)

65/65 [==============================] - 0s 4ms/step - loss: 0.1284 - accuracy: 0.9622


[0.1284310519695282, 0.9621726274490356]

In [ ]:
model_2.evaluate(X_test, y_test_encoded)

(2062, 21, 3)
(2062,)
65/65 [==============================] - 3s 50ms/step - loss: 0.1489 - accuracy: 0.9675


[0.14891590178012848, 0.9675073027610779]

# Save Model to JSON (for tensorflowjs)

In [ ]:
!pip install tensorflowjs

In [ ]:
!pip install jaxlib==0.4.2

In [ ]:
!pip install jaxlib==0.3.25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.2/71.2 MB 8.7 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.2
    Uninstalling jaxlib-0.4.2:
      Successfully uninstalled jaxlib-0.4.2


In [ ]:
!pip show jaxlib

In [ ]:
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    --output_format=tfjs_graph_model \
    --signature_name=serving_default \
    --saved_model_tags=serve \
    /content/my_model \
    /content/tfjs_model

2023-02-15 22:03:43.849606: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-15 22:03:45.903659: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-15 22:03:45.903845: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-15 22:03:45.903881: W tensorfl